In [1]:
import numpy as np
from scipy import linalg

In [ ]:
# Question (1): Calculating Norms

x = np.array([-9, 5, 1, -4, 12, -7])

print(f"{linalg.norm(x, 1)}, {linalg.norm(x, 2)}, {linalg.norm(x, np.inf)}")

# (Explanation). As expected.

38, 17.776388834631177
38.0, 17.776388834631177, 12.0


### Question (2): Tight Inequalities
A tight inequality is one such that there exists some choice of variable for which the inequality holds as an equality.

For each of the following,
- $||x||_2 \leq ||x||_1 \leq \sqrt{n} ||x||_2$,
- $||x||_\infty \leq ||x||_2 \leq \sqrt{n} ||x||_\infty$,
- $||x||_\infty \leq ||x||_1 \leq n ||x||_\infty$,

the choice $x = [1, 0, \ldots, 0]$ and $x = [1, 1, \ldots, 1]$ work.

In [ ]:
# Question (3)(a): Matrix Norms

A = np.array([
    [3, -2, 0, 5],
    [1,  8, 7, 0],
    [-9, 5, 1, 2],
    [3, -2, 0, 5],
    [3, -4, 0, 5]
])

print("||A||_1 =", linalg.norm(A, 1))
print("||A||_∞ =", linalg.norm(A, np.inf))
print("||A||_2 =", linalg.norm(A, 2))

# (Explanation). Use syntax as for column vectors.
#   - We use `np.array` so that we can slice later

||A||_1 = 21.0
||A||_∞ = 17.0
||A||_2 = 14.138407784747079


In [ ]:
# Question (3)(b): Matrix Calculations

B = A[:4, :]

print(f"{linalg.norm(B, 1)}, {linalg.norm(B, 2)}, {linalg.norm(B, np.inf)}")

eigs = linalg.eigvals(B)
print("|λ_max| =", max(abs(eigs)))
print("||B||_2 equals |λ_max|?", linalg.norm(B, 2) == max(abs(eigs)))

# (Explanation). Use syntax as shown.

17.0, 12.965395613366699, 17.0
|λ_max| = 12.495863904919725
||B||_2 equals |λ_max|? False


### Question (4): Condition Numbers

The condition number of a matrix is defined as:
- $\kappa(A) = ||A|| \cdot ||A^{-1}||$.

This quantity can be thought of as how much a perturbation to $b$ affects the solution $x$, with higher values indicating a higher effect. 

In [18]:
# Question (4): Condition Numbers

# (a) A = αI
a = 3.0
I = np.identity(4)
A = a * I
print("cond_1(A) =", linalg.norm(A, 1) * linalg.norm(linalg.inv(A), 1))
print("cond_2(A) =", linalg.norm(A, 2) * linalg.norm(linalg.inv(A), 2))
print("cond_inf(A) =", linalg.norm(A, np.inf) * linalg.norm(linalg.inv(A), np.inf))

# (b) Hilbert matrices (n = 5, 10, 15)
for n in [5, 10, 15]:
    H = linalg.hilbert(n)
    print(f"\nHilbert matrix (n={n}):")
    print("cond_1 =", linalg.norm(H, 1) * linalg.norm(linalg.inv(H), 1))
    print("cond_2 =", linalg.norm(H, 2) * linalg.norm(linalg.inv(H), 2))
    print("cond_inf =", linalg.norm(H, np.inf) * linalg.norm(linalg.inv(H), np.inf))

cond_1(A) = 1.0
cond_2(A) = 1.0
cond_inf(A) = 1.0

Hilbert matrix (n=5):
cond_1 = 943656.0000063397
cond_2 = 476607.2502457646
cond_inf = 943656.0000063629

Hilbert matrix (n=10):
cond_1 = 35356843615851.7
cond_2 = 16026019477413.043
cond_inf = 35356847610517.09

Hilbert matrix (n=15):
cond_1 = 1.221413576195944e+18
cond_2 = 4.380347090438315e+17
cond_inf = 1.041726976490338e+18


In [ ]:
# Question (5): Significant Figures and Relative Error

def sig_figs(kappa, b_digits):
    return np.floor(b_digits - np.log10(kappa))

def rel_err_needed(kappa, desired_digits):
    return 10 ** (-(desired_digits - np.log10(kappa)))

# (a)
print("Case i:", sig_figs(1.2e1 * 3.9e3, 8))
print("Case ii:", sig_figs(198.2 / 0.00031, 8))  # cond ≈ max_eig / min_eig
print("Case iii:", sig_figs(1 / 9.9010e-12, 8))

# (b) Required error in b to get 6-digit solution
print("Case i:", rel_err_needed(1.2e1 * 3.9e3, 6))
print("Case ii:", rel_err_needed(198.2 / 0.00031, 6))
print("Case iii:", rel_err_needed(1 / 9.9010e-12, 6))

# (Explanation). For Ax = b,
#   - relative error of x <= condition number * relative error of b
#   - a solution has n significant figures if its relative error is less than 10^{-n}
#       - if it is greater, say n=3 and the relative error = 10^{-2} then the last digit is 'uncertain'
#   - re-arranging gives relative error in b <= 10^{-n}/condition number

Case i: 3.0
Case ii: 2.0
Case iii: -4.0
Case i: 0.046799999999999974
Case ii: 0.6393548387096775
Case iii: 100999.89900010117


In [ ]:
# Question (6): Symmetry Checker

from numpy import maximum, any, finfo, float32, float64, ndarray

def symchk(A, tol=None):
    A = A.astype(float)
    if A.shape[0] != A.shape[1]: # not square
        return False
    if tol is None:
        # heuristic: 10 operations per n^2 entries chances to accumulate error
        tol = 10 * A.shape[0]**2 * finfo(A.dtype).eps
    if tol < 0:
        raise ValueError("Tolerance must be non-negative")
    diff = abs(A - A.T)
    bound = tol * maximum(abs(A), abs(A.T)) # scale tolerance with highest value in either matrix
    return not any(diff > bound)

In [25]:
from numpy import array
from numpy.random import randn

A = array([[1, 2], [2, 1]])
B = randn(3, 3)
C = randn(3, 4)

print("A symmetric?", symchk(A))
print("B symmetric?", symchk(B))
print("C symmetric?", symchk(C))  # Not square


A symmetric? True
B symmetric? False
C symmetric? False
